# K-평균(K-Means) 알고리즘을 이용한 붓꽃 품종 군집화

**학습 목표:**
1. 비지도 학습의 일종인 K-평균 군집화 알고리즘의 원리를 이해합니다.
2. 사이킷런의 `KMeans`를 사용하여 붓꽃(iris) 데이터를 품종별로 군집화합니다.
3. 군집화 성능을 평가하는 방법인 실루엣 분석(Silhouette Analysis)과 엘보우 기법(Elbow Method)에 대해 학습합니다.
4. PCA(주성분 분석)를 통해 군집화 결과를 시각화하여 분석합니다.

## 1. K-평균 (K-Means) 군집화란?

K-평균은 대표적인 **비지도 학습(Unsupervised Learning)** 알고리즘 중 하나로, 데이터의 특정 패턴이나 구조를 파악하여 여러 개의 **군집(Cluster)**으로 나누는 방법입니다. 정답(Label)이 없는 데이터를 다룰 때 유용합니다.

알고리즘은 **군집 중심(Centroid)**이라는 임의의 지점을 선택하고, 각 데이터 포인트를 가장 가까운 중심에 할당하는 과정을 반복하여 최적의 군집을 형성합니다.

![군집화.png](images\군집화.png)

### K-Means 알고리즘 수행 절차

`k`는 사용자가 직접 지정해야 하는 **하이퍼파라미터**로, 군집의 개수를 의미합니다.

1. **초기화(Initialization)**: `k`개의 군집 중심(Centroid)을 임의로 선택합니다.
2. **할당(Assignment)**: 각 데이터 포인트를 유클리드 거리를 기준으로 가장 가까운 중심점에 할당합니다.
3. **갱신(Update)**: 각 군집에 속한 데이터 포인트들의 평균값을 계산하여 중심점의 위치를 업데이트합니다.
4. **반복(Iteration)**: 중심점의 위치에 더 이상 변화가 없을 때까지 2번과 3번 과정을 반복합니다.

D3.js를 활용한 K-Means 시각화 자료: [Visualizing K-Means Clustering](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/)

![군집1.png](images\군집1.png)

### K-평균의 장단점

**장점:**
- 알고리즘이 간단하고 구현이 쉽습니다.
- 대용량 데이터에도 비교적 빠르게 동작합니다.

**단점:**
- **거리 기반 알고리즘**이므로 속성(Feature)의 개수가 매우 많을 경우 군집화 정확도가 떨어질 수 있습니다. (→ PCA와 같은 차원 축소 기법을 함께 사용하기도 합니다.)
- 반복 횟수가 많아지면 수행 시간이 느려질 수 있습니다.
- 사용자가 군집의 개수(`k`)를 사전에 지정해야 하는 어려움이 있습니다.
- 초기 중심점 위치에 따라 군집화 결과가 달라질 수 있습니다.

### 사이킷런 `KMeans` 클래스 소개

사이킷런은 K-평균 군집화를 위한 `KMeans` 클래스를 제공합니다.

```python
# KMeans 모델을 생성하고 학습시킵니다.
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3,       # 군집의 개수(k)를 3으로 설정
                init='k-means++', # 중심점 초기화 방식 ('k-means++'는 더 나은 초기 중심점을 선택하는 알고리즘)
                max_iter=300,       # 최대 반복 횟수
                random_state=0)     # 결과의 재현성을 위한 시드(seed) 값

kmeans.fit(iris_df) # 붓꽃 데이터로 모델 학습
```

학습이 완료되면, 각 데이터 포인트가 어떤 군집에 속했는지 확인할 수 있습니다. 이 결과를 실제 붓꽃 품종(target)과 비교하여 군집화가 얼마나 잘 되었는지 평가할 수 있습니다.

```python
# 각 데이터가 속한 군집 레이블 확인 (0, 1, 2 중 하나)
print(kmeans.labels_)

# 실제 품종(target)과 군집 결과(cluster)를 비교
iris_df['target'] = iris.target
iris_df['cluster'] = kmeans.labels_
iris_df.groupby(['target', 'cluster'])['sepal_length'].count()
```

## 2. 군집화 성능 평가

군집화는 정답이 없는 비지도 학습이므로, 결과가 얼마나 의미 있게 잘 나뉘었는지 평가하는 지표가 필요합니다. 좋은 군집화는 **군집 내 응집도(Compactness)**는 높고, **군집 간 분리도(Separation)**는 높은 상태를 의미합니다.

![군집의평가.png](images\군집의평가.png)

In [ ]:
# 분석에 필요한 라이브러리와 데이터를 로드합니다.
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.cluster import KMeans

# 붓꽃 데이터셋을 로드합니다.
iris = load_iris()

In [ ]:
# 붓꽃 데이터를 DataFrame으로 변환합니다.
# 원본 노트북의 관례에 따라 (cm)를 제외한 열 이름을 사용합니다.
iris_df = pd.DataFrame(data=iris.data, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
iris_df.head(3)

In [ ]:
# K-Means 모델을 생성하고 붓꽃 데이터에 대해 군집화를 수행합니다.
KM = KMeans(n_clusters=3, init='k-means++', max_iter=300, random_state=0)

# fit() 메서드는 데이터를 모델에 맞게 학습시킵니다.
KM.fit(iris_df)

In [ ]:
# labels_ 속성은 각 데이터 포인트가 어떤 군집에 할당되었는지 나타내는 배열을 반환합니다.
# 붓꽃 데이터는 총 150개이므로, 150개의 레이블이 생성됩니다. (0, 1, 2)
KM.labels_

In [ ]:
# 군집화 결과를 실제 붓꽃 품종(target)과 비교하여 얼마나 잘 분류되었는지 확인합니다.

# 원본 데이터의 품종(target) 정보를 DataFrame에 추가합니다.
iris_df['target'] = iris.target
# K-Means 군집화 결과(cluster)를 DataFrame에 추가합니다.
iris_df['cluster'] = KM.labels_

# groupby를 사용하여 실제 품종(target)과 군집(cluster)별로 데이터 개수를 확인합니다.
# target 0 (Setosa)은 cluster 1로 50개 모두 정확하게 군집화되었습니다.
# target 1 (Versicolor)은 cluster 0으로 47개, cluster 2로 3개가 군집화되었습니다.
# target 2 (Virginica)는 cluster 0으로 14개, cluster 2로 36개가 군집화되었습니다.
iris_df.groupby(['target', 'cluster'])['sepal_length'].count()

### 실루엣 분석 (Silhouette Analysis)

실루엣 분석은 군집화 성능을 평가하는 대표적인 방법 중 하나로, 각 데이터 포인트가 자신이 속한 군집과 얼마나 유사하고, 다른 군집과는 얼마나 다른지를 측정합니다.

**실루엣 계수(Silhouette Coefficient)**는 -1에서 1 사이의 값을 가지며, 각 데이터 포인트에 대해 계산됩니다.
- **1에 가까울수록**: 현재 군집과 잘 묶여 있고, 다른 군집과는 멀리 떨어져 있음을 의미합니다. (Good!)
- **0에 가까울수록**: 군집의 경계에 위치하고 있음을 의미합니다. (So so...)
- **-1에 가까울수록**: 잘못된 군집에 할당되었음을 의미합니다. (Bad!)

![실루엣분석.png](images\실루엣분석.png)

**좋은 군집화의 조건:**
1. 전체 실루엣 계수의 **평균값**이 1에 가까울수록 좋습니다.
2. 개별 군집의 실루엣 계수 평균값이 전체 평균과 크게 차이 나지 않고, 특정 군집의 값들이 유난히 낮지 않아야 합니다. (군집별 두께가 비슷해야 함)

In [ ]:
# 실루엣 분석을 위한 라이브러리를 임포트합니다.
from sklearn.metrics import silhouette_score, silhouette_samples

# 각 데이터 포인트의 실루엣 계수를 계산합니다.
# silhouette_samples(X, labels) : X(피처 데이터), labels(군집 결과)
score_samples = silhouette_samples(iris.data, iris_df['cluster'])

# 계산된 실루엣 계수를 DataFrame에 새로운 컬럼으로 추가합니다.
iris_df['silhouette_coeff'] = score_samples
iris_df.head()

In [ ]:
# 전체 데이터의 평균 실루엣 계수를 계산합니다.
# 약 0.55로, 나쁘지 않은 군집화 결과를 보입니다.
silhouette_score(iris.data, iris_df['cluster'])

### 실루엣 차트 (Silhouette Plot) 시각화

실루엣 차트는 각 군집에 속한 데이터들의 실루엣 계수를 막대그래프로 나타내어 군집화 결과를 직관적으로 파악할 수 있게 도와줍니다.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 실루엣 차트를 그리기 위한 코드입니다.
y_lower = 10
for i in range(3): # 군집 개수가 3개이므로 0, 1, 2에 대해 반복
  # 현재 군집(i)에 해당하는 데이터의 실루엣 계수만 추출하여 정렬합니다.
  cluster_i = score_samples[iris_df['cluster']==i]
  cluster_i.sort()
  
  # 현재 군집의 막대그래프가 그려질 y축 위치를 계산합니다.
  y_upper = y_lower + len(cluster_i)
  
  # 가로 막대그래프(barh)를 그립니다.
  plt.barh(range(y_lower, y_upper), cluster_i, label=f'Cluster {i}')
  
  # 다음 군집이 그려질 시작 위치를 업데이트합니다.
  y_lower = y_upper

# 전체 평균 실루엣 점수를 빨간색 점선으로 표시합니다.
avg_score = silhouette_score(iris.data, iris_df['cluster'])
plt.axvline(avg_score, color='red', linestyle='--', label='Avg Score')

plt.xlabel('Silhouette Coefficient')
plt.ylabel('Cluster Label')
plt.legend()
plt.show()

# 그래프를 보면 k=3에서 팔꿈치 모양으로 꺾이는 것을 확인할 수 있으며, 따라서 k=3이 최적의 군집 수라고 판단할 수 있습니다.

## 3. 최적의 군집 수(k) 찾기

K-평균 알고리즘의 가장 큰 숙제는 최적의 `k`값을 찾는 것입니다. 이를 위해 다음과 같은 방법들이 사용됩니다.

| 방법 | 설명 |
| :--- | :--- |
| **엘보우(Elbow) 기법** | 군집 수를 늘려가면서 SSE(군집 내 오차 제곱합)의 변화를 관찰하여, 기울기가 급격히 완만해지는 '팔꿈치' 지점을 최적의 k로 선택합니다. |
| **실루엣 분석** | 여러 k값에 대한 평균 실루엣 계수를 계산하여, 가장 높은 값을 가지는 k를 선택합니다. |
| **Gap Statistic** | 실제 데이터와 랜덤 데이터 간의 성능 차이를 비교하여 통계적으로 유의미한 k를 찾습니다. |

### 엘보우 기법 (Elbow Method)

엘보우 기법은 군집의 개수(`k`)를 1부터 점차 늘려가면서 각 `k`에 대한 **SSE(Sum of Squared Errors, 오차제곱합)** 값을 계산하고, 이 값을 그래프로 그려 최적의 `k`를 찾는 방법입니다.

- **SSE**: 각 데이터 포인트와 자신이 속한 군집의 중심점 사이의 거리 제곱의 합입니다. SSE가 작을수록 군집 내 데이터들이 중심에 잘 모여있음을 의미합니다.

그래프에서 SSE 값이 급격히 감소하다가 완만해지는 지점, 즉 **'팔꿈치(Elbow)'**처럼 꺾이는 부분이 최적의 `k`값이 됩니다.

![엘보우기법.png](images\엘보우기법.png)

사이킷런의 `KMeans` 객체는 학습 후에 `inertia_` 속성을 통해 SSE 값을 제공합니다.

In [ ]:
# k=3일 때의 SSE(오차제곱합) 값을 확인합니다.
# inertia_ 속성은 군집 내 데이터들이 중심으로부터 얼마나 떨어져 있는지를 나타냅니다.
KM.inertia_

In [ ]:
# 엘보우 기법을 적용하기 위해 원본 피처 데이터만 다시 준비합니다.
scree_df = iris_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

In [ ]:
# k값을 1부터 10까지 변화시키면서 각 k에 대한 SSE(inertia)를 저장할 리스트입니다.
inertia = []
k_values = range(1, 11)

for k in k_values:
  # 각 k값에 대해 KMeans 모델을 생성하고 학습시킵니다.
  kmeans = KMeans(n_clusters=k, n_init=10, random_state=42) # n_init='auto' in newer versions
  kmeans.fit(scree_df)
  # 계산된 SSE(inertia) 값을 리스트에 추가합니다.
  inertia.append(kmeans.inertia_)

# k값의 변화에 따른 SSE 값을 꺾은선 그래프로 시각화합니다.
plt.plot(k_values, inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia (SSE)')
plt.xticks(k_values)
plt.grid(True)
plt.show()

# 그래프를 보면 k=3에서 팔꿈치 모양으로 꺾이는 것을 확인할 수 있으며, 따라서 k=3이 최적의 군집 수라고 판단할 수 있습니다.

## 4. PCA를 이용한 군집 결과 시각화

붓꽃 데이터는 4개의 속성(4차원)을 가지고 있어 직접 시각화하기 어렵습니다. **PCA(Principal Component Analysis, 주성분 분석)**를 사용하여 데이터를 2차원으로 축소하면, 군집화 결과를 산점도(Scatter Plot)로 쉽게 시각화할 수 있습니다.

In [ ]:
# PCA를 위한 라이브러리를 임포트합니다.
from sklearn.decomposition import PCA

# n_components=2 : 데이터를 2개의 주성분(2차원)으로 축소하도록 설정합니다.
pca = PCA(n_components=2)

# fit_transform()을 사용하여 붓꽃 데이터를 2차원으로 변환합니다.
pca_transformed = pca.fit_transform(iris.data)

# 변환된 데이터의 차원을 확인합니다. (150개의 데이터, 2개의 차원)
pca_transformed.shape

In [ ]:
# PCA 변환 결과를 DataFrame에 새로운 컬럼으로 추가합니다.
iris_df['pca_x'] = pca_transformed[:, 0] # 첫 번째 주성분
iris_df['pca_y'] = pca_transformed[:, 1] # 두 번째 주성분
iris_df.head()

### PCA 결과 시각화

2차원으로 축소된 데이터를 산점도로 시각화하여 군집들이 어떻게 분포되어 있는지 확인합니다.

In [ ]:
# 각 군집별로 다른 마커(o, ^, s)를 사용하여 산점도를 그립니다.
plt.scatter(x=iris_df.loc[iris_df['cluster']==0, 'pca_x'], 
            y=iris_df.loc[iris_df['cluster']==0, 'pca_y'], marker='o', label='Cluster 0')
plt.scatter(x=iris_df.loc[iris_df['cluster']==1, 'pca_x'], 
            y=iris_df.loc[iris_df['cluster']==1, 'pca_y'], marker='^', label='Cluster 1')
plt.scatter(x=iris_df.loc[iris_df['cluster']==2, 'pca_x'], 
            y=iris_df.loc[iris_df['cluster']==2, 'pca_y'], marker='s', label='Cluster 2')

plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Clusters Visualization with PCA')
plt.legend()
plt.show()

# 시각화 결과, Cluster 1(Setosa)은 명확하게 분리되어 있고, Cluster 0과 2는 일부 겹치는 것을 확인할 수 있습니다.

### [심화] 데이터 스케일링 후 군집화

K-평균은 거리를 기반으로 동작하므로, 각 피처(속성)의 스케일(척도)이 다를 경우 분석 결과가 왜곡될 수 있습니다. 예를 들어, 한 피처는 0~1 사이의 값을 갖고 다른 피처는 0~1000 사이의 값을 갖는다면, 후자의 영향력이 훨씬 커지게 됩니다.

**StandardScaler**를 사용하여 모든 피처를 평균 0, 분산 1인 표준 정규 분포로 스케일링한 후 군집화를 수행하면 더 안정적인 결과를 얻을 수 있습니다.

In [ ]:
# StandardScaler를 임포트합니다.
from sklearn.preprocessing import StandardScaler

# 붓꽃 데이터를 표준 정규 분포로 스케일링합니다.
scaler = StandardScaler()
iris_scaled = scaler.fit_transform(iris_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

# 스케일링된 데이터로 K-Means 군집화를 다시 수행합니다.
kmeans_scaled = KMeans(n_clusters=3, n_init=10, random_state=42)
kmeans_scaled.fit(iris_scaled)

# 스케일링 후의 군집화 결과를 DataFrame에 추가합니다.
iris_df['scaled_cluster'] = kmeans_scaled.labels_

# 스케일링 전후의 군집화 결과를 비교합니다.
print('Original Clustering Results:')
print(iris_df.groupby(['target', 'cluster'])['sepal_length'].count())
print('
Scaled Clustering Results:')
print(iris_df.groupby(['target', 'scaled_cluster'])['sepal_length'].count())

# 스케일링 후 Versicolor(target=1)와 Virginica(target=2)의 군집화 성능이 약간 향상된 것을 볼 수 있습니다.

## 정리

이번 시간에는 K-평균 군집화 알고리즘을 사용하여 붓꽃 데이터를 품종별로 그룹화하는 과정을 학습했습니다.

1. **K-평균 알고리즘**: 비지도 학습으로, 데이터를 `k`개의 군집으로 나누는 방법입니다.
2. **최적의 k 찾기**: **엘보우 기법**을 사용하여 SSE의 변화를 관찰하고, 그래프가 꺾이는 지점인 `k=3`을 최적의 군집 수로 결정했습니다.
3. **군집 성능 평가**: **실루엣 분석**을 통해 군집이 얼마나 잘 형성되었는지 평가했습니다. 평균 실루엣 점수가 약 0.55로 나타났습니다.
4. **시각화**: 4차원의 붓꽃 데이터를 **PCA**를 이용해 2차원으로 축소하여, 군집화 결과를 산점도로 시각화하고 분포를 확인했습니다.
5. **데이터 스케일링**: 거리 기반 알고리즘의 성능 향상을 위해 **StandardScaler**로 데이터를 정규화하는 과정의 중요성을 확인했습니다.

이러한 군집화 기법은 고객 분류, 이상 탐지, 이미지 분할 등 다양한 분야에서 활용될 수 있습니다.

In [ ]:
# pca 2개의주성분도 예시 작성